In [55]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [56]:
data_path="../../PlantVillage/"
image_size=tuple((250,250))
batch_size=16
epochs=10

In [57]:
!ls {data_path}

Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___Late_blight
Potato___healthy
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_Late_blight
Tomato_Leaf_Mold
Tomato_Septoria_leaf_spot
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato__Target_Spot
Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato__Tomato_mosaic_virus
Tomato_healthy


In [58]:
def convert_resize_image(image):
    image=cv2.imread(image)
    image=cv2.resize(image, image_size)
    return img_to_array(image)    

In [75]:
images=[]
labels=[]

plant_disease_folders = os.listdir(f"{data_path}")

for plant_disease_folder in plant_disease_folders:
    plant_disease_images = os.listdir(f"{data_path}/{plant_disease_folder}/")
    
    for image in plant_disease_images[:70]:
        image_dir = f"{data_path}/{plant_disease_folder}/{image}"
        images.append(convert_resize_image(image_dir))
        labels.append(plant_disease_folder)
print(images)
 

[array([[[ 97.,  94., 109.],
        [ 92.,  89., 104.],
        [120., 117., 133.],
        ...,
        [149., 148., 164.],
        [154., 153., 169.],
        [164., 163., 179.]],

       [[126., 123., 138.],
        [101.,  98., 113.],
        [118., 115., 130.],
        ...,
        [149., 148., 164.],
        [148., 147., 163.],
        [151., 150., 166.]],

       [[133., 130., 145.],
        [ 90.,  87., 102.],
        [141., 138., 153.],
        ...,
        [153., 152., 168.],
        [147., 146., 162.],
        [145., 144., 160.]],

       ...,

       [[150., 143., 156.],
        [166., 159., 172.],
        [205., 198., 211.],
        ...,
        [152., 143., 156.],
        [152., 143., 156.],
        [151., 142., 155.]],

       [[155., 148., 161.],
        [136., 129., 142.],
        [144., 137., 150.],
        ...,
        [165., 156., 169.],
        [167., 158., 171.],
        [165., 156., 169.]],

       [[172., 165., 177.],
        [162., 155., 167.],
        [190., 

In [76]:
label_encoder=LabelEncoder()
label_binarizer=LabelBinarizer()

In [77]:
image_labels_enc=label_encoder.fit_transform(labels)
image_labels_bin=label_binarizer.fit_transform(labels)

In [78]:
labels

['Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell___Bacterial_spot',
 'Pepper__bell

In [79]:
image_labels_enc

array([ 0,  0,  0, ..., 12, 12, 12], dtype=int64)

In [80]:
image_labels_bin[11]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
label_binarizer.classes_

array(['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy',
       'Potato___Early_blight', 'Potato___Late_blight',
       'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
       'Tomato_Late_blight', 'Tomato_Leaf_Mold',
       'Tomato_Septoria_leaf_spot',
       'Tomato_Spider_mites_Two_spotted_spider_mite',
       'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
       'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'], dtype='<U43')

In [82]:
normalized_images=np.array(images) /225.0

In [83]:
normalized_images

array([[[[0.4311111 , 0.41777778, 0.48444444],
         [0.40888888, 0.39555556, 0.46222222],
         [0.53333336, 0.52      , 0.5911111 ],
         ...,
         [0.6622222 , 0.6577778 , 0.72888887],
         [0.6844444 , 0.68      , 0.7511111 ],
         [0.72888887, 0.72444445, 0.79555553]],

        [[0.56      , 0.5466667 , 0.61333334],
         [0.4488889 , 0.43555555, 0.50222224],
         [0.52444446, 0.51111114, 0.5777778 ],
         ...,
         [0.6622222 , 0.6577778 , 0.72888887],
         [0.6577778 , 0.6533333 , 0.72444445],
         [0.6711111 , 0.6666667 , 0.73777777]],

        [[0.5911111 , 0.5777778 , 0.64444447],
         [0.4       , 0.38666666, 0.45333335],
         [0.62666667, 0.61333334, 0.68      ],
         ...,
         [0.68      , 0.6755555 , 0.74666667],
         [0.6533333 , 0.6488889 , 0.72      ],
         [0.64444447, 0.64      , 0.7111111 ]],

        ...,

        [[0.6666667 , 0.63555557, 0.6933333 ],
         [0.73777777, 0.70666665, 0.7644445 ]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(normalized_images, image_labels_bin, test_size=0.3, random_state=42)

In [85]:
train_datagen = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,horizontal_flip=True)

In [86]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3, 250, 250) 
    batch_norm_axis=1
else: 
    input_shape = (250,250, 3)
    batch_norm_axis=-1

In [87]:
model = Sequential() 
model.add(Conv2D(32, (3, 3),padding="same",input_shape=input_shape))
model.add(Activation('relu')) 
model.add(BatchNormalization(axis=batch_norm_axis))
model.add(MaxPooling2D(pool_size=(3, 3))) 
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3),padding="same")) 
model.add(Activation('relu')) 
model.add(Conv2D(64, (3, 3),padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=batch_norm_axis))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3),padding="same"))
model.add(Activation('relu')) 
model.add(Conv2D(128, (3, 3),padding="same"))
model.add(Activation('relu')) 
model.add(BatchNormalization(axis=batch_norm_axis))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten()) 
model.add(Dense(1000)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(label_binarizer.classes_))) 
model.add(Activation('softmax')) 

In [88]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 250, 250, 32)      896       
_________________________________________________________________
activation_21 (Activation)   (None, 250, 250, 32)      0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 250, 250, 32)      128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 83, 83, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 83, 83, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 83, 83, 64)        18496     
_________________________________________________________________
activation_22 (Activation)   (None, 83, 83, 64)       

In [89]:
optimizer=Adam(lr=0.001, decay=0.001/ 3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
fitter=model.fit_generator(train_datagen.flow(X_train, y_train, batch_size=batch_size), validation_data=(X_test, y_test),steps_per_epoch=len(X_train) // batch_size,epochs=3, verbose=1) 

Epoch 1/3
 2/45 [>.............................] - ETA: 9:02 - loss: 1.0950 - acc: 0.8792 